In [ ]:
!pip install numpy
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

# ----------------------------
# 1. Generate Synthetic Meteorological Data
# ----------------------------
np.random.seed(42)
n_samples = 1000
image_size = (10, 10)  # 10x10 grid of "sensors"
n_channels = 3         # e.g., temp, pressure, wind

# Create synthetic 3D data: [samples, height, width, channels]
X = np.random.rand(n_samples, image_size[0], image_size[1], n_channels)

# Simulate labels: 1 = extreme event, 0 = normal
# Use a simple rule: if sum of "pressure + temp" > threshold, it's extreme
threshold = 15.0
pressure_temp_sum = X[:, :, :, 0] + X[:, :, :, 1]
event_trigger = pressure_temp_sum.mean(axis=(1, 2)) > threshold
y = event_trigger.astype(int)

# ----------------------------
# 2. Split Data
# ----------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ----------------------------
# 3. Build CNN Model
# ----------------------------
model = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(10, 10, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

# ----------------------------
# 4. Train Model
# ----------------------------
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# ----------------------------
# 5. Evaluate Model
# ----------------------------
y_pred = (model.predict(X_test) > 0.5).astype(int).flatten()
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# ----------------------------
# 6. Plot Training History
# ----------------------------
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training History')
plt.grid(True)
plt.tight_layout()
plt.show()
